In [ ]:
import cv2
import numpy as np
import os
import glob
# チェッカーボードの定義（チェッカーボードの行と列にある内部コーナーの数）
CHECKERBOARD = (4,7) 

# コーナー検出のための反復アルゴリズムの終了条件
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# 各チェッカーボード画像の3Dポイントを保存する配列
objpoints = []
# 各チェッカーボード画像の2Dポイントを保存する配列
imgpoints = [] 
# 3Dポイント座標定義
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

# 指定されたディレクトリにある画像のパスを取得します
images = glob.glob('./ConvertedIMG/*.jpg')
for fname in images:
    img = cv2.imread(fname)
    # グレースケールに変換
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # コーナー検出 ret = true
    ret, corners = cv2.findChessboardCorners(gray,
                                             CHECKERBOARD,
                                             cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    # コーナーが検出された場合、以下の処理を行う
    if ret == True:
        objpoints.append(objp)

        # 検出されたコーナーの精度を向上
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        imgpoints.append(corners2)

        # コーナーを画像に描画し、表示
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    cv2.imshow('img',img)
    cv2.waitKey(0)
cv2.destroyAllWindows()
h,w = img.shape[:2] # 480, 640
# カメラキャリブレーションを行い、カメラマトリックス、歪み係数などを取得
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
print("Camera matrix : \n") #カメラマトリックス
print(mtx)

In [ ]:
print("dist : \n") #レンズ歪み
print(dist)

In [ ]:
print("rvecs : \n") #回転ベクトル
print(rvecs)

In [ ]:
print("tvecs : \n") #移動ベクトル
print(tvecs)

In [ ]:
np.save("mtx", mtx) # カメラ行列セーブ
np.save("dist", dist.ravel()) # 歪みパラメータセーブ

In [ ]:
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1)
dst2 = cv2.undistort(img, mtx, dist, None, newcameramtx) ## 適用後のイメージ
cv2.imshow('num2', dst2)
cv2.waitKey(0)
cv2.destroyAllWindows()